In [1]:
import numpy as np
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *

In [2]:
#Importación de librerías

#!pip install qiskit ipywidgets    correr esta línea de primero si se trabaja en google collab
#Importación de qiskit
from qiskit import *

%matplotlib inline
from matplotlib import *

#Se importa quantum_info para poder trabajar con herramientas útiles como las matríces de pauli y elementos para trabajar
# con operadores.
import qiskit.quantum_info as qi
from qiskit.quantum_info import *

#Simplemente para que no se vean los warning
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

#Librerías matemáticas importantes
import numpy as np
import matplotlib.pyplot as plt

# Energía para el modelo de Ising

El Hamiltoniano que describe al modelo de Ising se puede ver mediante la siguiente ecuación
$$ H = \sum_i^n \sigma_i^z \sigma_{i+1}^z+  g \sum_i^{n+1} \sigma_i^x   .$$

Entonces, es posible encontrar el valor promedio del hamiltoniano de la forma
$$  < \psi| H | \psi> = \sum_i^n <\psi|\sigma_i^z \sigma_{i+1}^z|\psi>  + g\sum_i^{n+1}<\psi| \sigma_i^x |\psi> $$

que es el nivel de energía correspondiente a esa función de onda $| \psi >$

El siguiente código busca esa implementación

In [3]:
def energia_ising(qubits,statevec,g):
    """  Energia para el Hamiltoniano del modelo de Ising para una función de onda:
    
         Esta función calcula la energía o valor promedio del Hamiltoniano perteneciente al 
         modelo de Ising para una función de estado otorgada y una constante g también 
         arbitraria. 
    
    Entrada(s): 
        statevec: vector de estado correspondiente a n-qubits que se están tratando. 
        g= constante real entera g como parte del Hamiltoniano
    
    """
    energia=0
    
    #Se itera por cada qubit hasta n-1 (sumatoria en sigmaz sigmaz+1)
    for i in range(0,qubits-1):
        
        parop=['','','']                   
        
        #Iteración sobre cada i-ésimo agregando la Identidad en el producto tensorial para la sumatoria de sigmaz,sigmax
        for j in range(0,i):
            parop[0]= parop[0] + 'I'
            parop[1]= parop[1] + 'I'

        #Se agrega ZZ como X en cada iteración
        parop[0] = parop[0] + 'ZZ'
        parop[1] = parop[1] + 'X'

        
        #Se agrega el tensor con la identidad para completar la dimensión del esatdo
        for w in range(i+2,qubits):
            parop[0] = parop[0]+'I'
        for q in range(i+1,qubits):
            parop[1] = parop[1]+'I'
            
            
        #Se encuentra el valor de energía para la i-ésima iteración, se suma al valor anterior
        energia= energia + np.matmul( statevec,  np.matmul(qi.Pauli(parop[0]).to_matrix() , statevec) ) + g*np.matmul(statevec, np.matmul(qi.Pauli(parop[1]).to_matrix() , statevec) )
        
        #Ya que la sumatoria recorre la de sigmaz, falta agregar un último término de sigmax,aquí se agrega
        if i==qubits-2:
            for k in range(0,i+1):
                parop[2]= parop[2] + 'I'
            parop[2]= parop[2]+ 'X'
            energia = energia + g*np.matmul(statevec, np.matmul(qi.Pauli(parop[2]).to_matrix(),statevec) ) #Se suma a la función de costo
        
          #Debugging
#         print(parop[0],parop[1],parop[2])  
#         print(energia)
        
    return energia

In [4]:
#Si por ejeemplo se tiene un estado que representa a 2 qubits (este caso no esta normalizado, tomarlo en cuenta)
#el código nos regresará su energía correspondiente
statevec=np.array([1,2,3,4])
energia_ising(2,statevec,2)

(104+0j)

In [4]:
# Marco - podemos verificar cual es la menor energia calculando los autovalores del Hamiltoniano. 
# Vemos que el autovalor mas pequenio es -1.41421356

H = S_zz + g*(S_x_1+S_x_2)

numpy.linalg.eig(H)

KeyboardInterrupt: 

# Generador de circuitos para n qubits

Definiremos los parámetros theta que son interpretados como rotaciones en un circuito en el eje y, estos caracterizan por un vector $\vec{\theta}$, por lo que nuestro circuito dependerá de ese parámtetro. El código siguiente busca esa implementación

In [251]:
def circ_nqubits(qubits,capas,parametros):
  """ Circuito para modelo de Ising:
  
      La función circ_nqubits genera un circuito para un número n de qubits
      con un número n de capas que el usuario desee implementar. Para esto es necesaria
      una matriz con los parámetros theta de dimensión  qubits x capas. 
      
  Entrada(s):
      qubits: el número de qubits a analizar. 
      capas: el número de capas para el ansatz del circuito. 
      parámetros: vector de dimension capasxparametros conteniendo rotaciones.  
  
  Salida(s): 
      circ: objeto de tipo QuantumCircuit conteniendo las instrucciones del circuito cuántico para IBM, como Qiskit.
      Statevector: vector de estado resultante del circuito.  
  
  
  """
  parametros = np.transpose(np.reshape(parametros,(capas,qubits)))  #Se escribe nuestro vector de parámetros como una matriz. 
  circ=QuantumCircuit(qubits)              #Inicilización del circuito
  
 
  capflag=True                             #Condición inicial de la bandera de cada capa

  for i in range(0,capas):
        
        flag1=True                         #Condiciones iniciales de las banderas para los cx
        flag2=False
        
        
        #Rotaciones parametrizadas al inicio de la capa
        for j in range(0,qubits):
             circ.ry(parametros[j][i],[j]) #Se aplica una rotación de las componentes de la matríz 
        
        
        #Revisión para la capa con bandera verdadera
        if capflag==True:
            #Se aplicará la cX a cada par de qubits, dejando uno como espacio. 
            for j in range(0,qubits-1):
                  if flag1==True: 
                    circ.cx([j],[j+1])     #Aplica la cX entre los 2 próximos
                    flag1=False            #Baja la bandera, así en la proxima iteración osalta el j+1

         #Salto de un qubit para el cx           
                  elif flag1==False:
                    flag1=True             #Sube la bandera para que en la siguiente si se aplique el cX 
            capflag=False                  #Se baja la bandera de la capa para que se use el cx entre los restantes
         
        #Revisión para la capa con bandera falsa (cx para unitr los qubits)
        elif capflag==False:
            #Se aplicará cx hacia los pares de qubits que todavía no se han conectado
            for w in range(0,qubits-1):
                if flag2==True:
                    circ.cx([w],[w+1])     #Aplica cx entre los 2 próximos al igual que la anterior
                    flag2=False            #Baja la bandera, en la próxima iteración saltará la cx entre esos qubits
                    
                elif flag2==False:
                    flag2=True             #Se sube la bandera para que no se aplique
                   
            capflag=True                   #Se retorna la bandera de la capa como verdadera nuevamente
            
           
  #Extraemos el statevector del circuito directamente     
  statevec=Statevector.from_instruction(circ).__array__()
  return statevec  
    

In [252]:
#Como ejemplo para este circuito se tienen ciertos parametros para 3 qubits con 2 capas
parametros=[1,2,3,4,5,6]

circ_nqubits(3,2,parametros)


array([-0.16375574+0.j,  0.11557204+0.j, -0.09508431+0.j,  0.28742526+0.j,
       -0.75938973+0.j,  0.53594594+0.j, -0.0205041 +0.j,  0.06198074+0.j])

# Función de costo

La función de costo busca analizar el comportamiento de los valores de energía para los parámetros $\vec \theta$ que se estén pasando. Ella se peude ver de la forma

$$ C(\theta) = < \psi(\theta)| H | \psi (\theta) > .$$

Para obtenerla entonces ahora basta con unir las dos implementaciones anteriores resultando el el siguiente código

In [253]:
#Función de costo
def cost_function(parametros):
    qubits=3   #Se designa el número de qubits
    g=0.5      #Se mira el valor de g
    capas=3    #Se establece el número de capas
    
    #Se ejecutan las dos funciones anteriores y se obtiene el valor de la función de
    # costo para este valor. 
    statevec=circ_nqubits(qubits,capas,parametros)
    
    # Calculamos la funcion de costo para ese set de parametros
    cost_value=energia_ising(qubits,statevec,g)
  
    return(cost_value.real)

In [254]:
# Marco - Ahora tenemos que entrenar los parametros, para esto usamos un optimmizador de python, por ejemplo

import scipy

#Condiciones iniciales
parame2=np.random.rand(9,1)*2*np.pi
scipy.optimize.minimize(cost_function,parame2,method='COBYLA',options={'maxiter':300})


     fun: -2.3192235156781327
   maxcv: 0.0
 message: 'Maximum number of function evaluations has been exceeded.'
    nfev: 300
  status: 2
 success: False
       x: array([[5.82697113],
       [3.38688546],
       [6.10694859],
       [5.32641472],
       [6.49373025],
       [4.70726894],
       [3.67042956],
       [7.13160875],
       [5.08485954]])